In [1]:
import os
import sys

# Ensure the current directory is in the Python path
sys.path.append(r"..\utils")
from loader import ImageDataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler
import numpy as np
from pathlib import Path

In [2]:
csv_dir = '../data/train_csv/train_file_list.csv'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(csv_dir=csv_dir, transform=transform)

# DataLoader 생성
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,  # 코어 수에 맞게 조정
    pin_memory=True,
    prefetch_factor=2
)

for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [3]:
def split_dataset(dataset, train_ratio=0.8):
    train_size = int(train_ratio * len(dataset))
    val_size = len(dataset) - train_size
    return torch.utils.data.random_split(dataset, [train_size, val_size])

In [4]:
train_dataset, val_dataset = split_dataset(dataset)

In [5]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=2
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    prefetch_factor=2
)

In [6]:
for images, labels in train_dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [7]:
for images, labels in val_dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [8]:
# fine tunning
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

In [9]:
def train(
    model,
    train_loader,
    val_loader=None,
    epochs=10,
    learning_rate=0.001,
    device=None,
    save_dir='checkpoints'
):
    # 1. Device 설정
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # 2. 디렉토리 생성
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    # 3. 최적화 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = GradScaler()  # Mixed Precision
    
    # 4. 메트릭 트래킹
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # 5. Train Phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_progress = tqdm(train_loader, desc=f'Train Epoch {epoch+1}/{epochs}', leave=False)
        for images, labels in train_progress:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            # 6. Mixed Precision Training
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # 7. 역전파 최적화
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient Clipping
            scaler.step(optimizer)
            scaler.update()
            
            # 8. 메트릭 계산
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            train_progress.set_postfix({
                'loss': running_loss/(train_progress.n+1),
                'acc': 100.*correct/total
            })
        
        # 9. Validation Phase
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        if val_loader:
            model.eval()
            val_progress = tqdm(val_loader, desc=f'Val Epoch {epoch+1}/{epochs}', leave=False)
            with torch.no_grad():
                for images, labels in val_progress:
                    images, labels = images.to(device), labels.to(device)
                    
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_correct += predicted.eq(labels).sum().item()
                    val_total += labels.size(0)
                    
                    val_progress.set_postfix({
                        'val_loss': val_loss/(val_progress.n+1),
                        'val_acc': 100.*val_correct/val_total
                    })
        
        # 10. 기록 저장
        train_loss = running_loss/len(train_loader)
        train_acc = 100.*correct/total
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        if val_loader:
            val_loss = val_loss/len(val_loader)
            val_acc = 100.*val_correct/val_total
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            
            # 11. Best Model 저장
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), f'{save_dir}/best_model.pth')
        
        # 12. 학습률 조정
        scheduler.step()
        
        # 13. Epoch 결과 출력
        log = f"Epoch [{epoch+1}/{epochs}] | "
        log += f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%"
        if val_loader:
            log += f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%"
        print(log)
    
    # 14. 최종 모델 저장
    torch.save(model.state_dict(), f'{save_dir}/final_model.pth')
    return history


In [10]:
torch.cuda.empty_cache()

In [11]:
# torch available
torch.cuda.is_available()

True

In [12]:
# 학습 실행
history = train(
    model=model,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    epochs=50,
    learning_rate=1e-3,
    save_dir='experiment1'
)

Train Epoch 1/50:   0%|          | 0/829 [00:15<?, ?it/s]

Val Epoch 1/50:   0%|          | 0/208 [00:15<?, ?it/s]

Epoch [1/50] | Train Loss: 2.5361 | Train Acc: 46.94% | Val Loss: 0.6906 | Val Acc: 79.45%


Train Epoch 2/50:   0%|          | 0/829 [00:15<?, ?it/s]

Val Epoch 2/50:   0%|          | 0/208 [00:15<?, ?it/s]

Epoch [2/50] | Train Loss: 0.5824 | Train Acc: 83.25% | Val Loss: 0.4448 | Val Acc: 86.93%


Train Epoch 3/50:   0%|          | 0/829 [00:15<?, ?it/s]

Val Epoch 3/50:   0%|          | 0/208 [00:15<?, ?it/s]

In [14]:
import torchsummary
import torch
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet50(weights=None)
checkpoint_path = r'C:\works\dacon\img_clf\experiment1\best_model.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint, strict=False)
model.to(device)
model = resnet50(weights=None)
torch.load(checkpoint_path, map_location=device)

OrderedDict([('conv1.weight',
              tensor([[[[ 4.2616e-02,  2.0471e-02,  6.2883e-02,  ...,  5.9410e-02,
                         -1.8979e-02,  2.8901e-02],
                        [-2.2983e-02,  9.7923e-02,  1.2278e-01,  ...,  9.0346e-02,
                          3.1831e-02, -3.5528e-02],
                        [ 9.3836e-02, -2.6467e-01,  4.5918e-01,  ..., -1.4578e-01,
                          2.3261e-01, -7.0474e-02],
                        ...,
                        [-3.5828e-02,  4.0185e-01, -4.5217e-01,  ...,  7.4906e-01,
                         -5.7475e-01,  2.3904e-01],
                        [ 4.3234e-02, -1.9273e-01,  6.3773e-01,  ...,  5.4745e-01,
                         -8.6881e-03, -5.9861e-02],
                        [ 5.8915e-02, -1.4124e-01,  1.8492e-02,  ..., -3.3589e-01,
                          2.2456e-01,  3.9913e-02]],
              
                       [[ 2.2363e-02, -1.2316e-03, -1.8565e-02,  ...,  8.0434e-02,
                         -7.9473

In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
torchsummary.summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [1]:
# read config
import json

with open("/Users/iyongjeong/WORK/dacon/img_clf/config.json", 'r') as f:
    config = json.load(f)

In [5]:
print(config['loss_weight'])

In [8]:
if config['loss_weight'] == "":
    print("yes")
else:
    print("NO")

yes
